# Pre-Processing Data, Version B

Using Large Language Models to add much needed context to interviews.

This is a pre-processing document that prepares the Boder 2020 testimonies for pronoun disambiguation and context rewriting.

What is different from Version A:



Authors: Billy Peir with ideas from Ulysses Pascal

# Import Openai LLM





In [ ]:
#@title Step 1: Install Libraries

!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.1/456.1 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.9
    Uninstalling openai-1.59.9:
      Successfully uninstalled openai-1.59.9


### Step 2: Generate and Set Open AI Access Token

* Accept the inviation to Join the Organization
* Login to openai.com
* go to https://platform.openai.com/api-keys
* Generate a Project API Key
* Copy the API Key
* In Google Colab: Click the "Key" icon on the left.
* Create colab notebook access token named `OPEN_AI_PROJECT_KEY`
* Copy and Paste the Access Token from Open AI under `Value`.
* Make sure `Notebook Access` is turned `On`.

This process allows you to access models from Open AI




In [ ]:
#@title Step 3: Authenticate Notebook's Access

# Import Colab Secrets userdata module
from google.colab import userdata
import os

#Set the environment variable to the access token
os.environ["OPEN_AI_KEY"] = userdata.get('OPEN_AI_PROJECT_KEY')

In [ ]:
#@title Step 4: Define Generic Prompting Function.
from openai import OpenAI
from pydantic import BaseModel, Field
from pydantic import TypeAdapter
from typing import List
from textwrap import dedent
import json

# Pass the API key to Open AI
client = OpenAI(api_key=os.environ.get("OPEN_AI_KEY"))


class ResponseTemplate(BaseModel):
  response: str = Field(
        ...,
        description="the response"
    )

# Define a wrapper function to call Open AI
def prompt_gpt(prompt,
               model = "gpt-4o-mini",
               max_tokens = 4096,
               temperature = 0.9,
               top_p = 0.9,
               response_format= ResponseTemplate,
               frequency_penalty = 0.2,
               presence_penalty = 0.0,
               system="Your role is to process data factually without generating hallucinations or fabrications"):
    '''
    Wrapper for querying the GPT API with a single function.

    Args:
        prompt (str): The query sent to the model API.
        model (str, optional): The version of GPT to be queried. Defaults to "gpt-4o-mini".
        max_tokens (int, optional): The maximum number of tokens to generate. Defaults to 4096.
        temperature (float, optional): The sampling temperature to use. Defaults to 0.9.
        top_p (float, optional): The nucleus sampling parameter. Defaults to 0.1.
        frequency_penalty (float, optional): The penalty for token frequency. Defaults to 0.2.
        presence_penalty (float, optional): The penalty for token presence. Defaults to 0.0.
        system (str, optional): The system-level prompt that defines the model's role.

    Returns:
        str: The text response generated by the model.
    '''
    response = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": dedent(prompt)}
        ],
        response_format= response_format,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
    )

    output = response.choices[0].message.parsed

    # Dump the model to a dictionary with JSON-compatible formatting
    output_dict = output.model_dump(mode='json')

    # Return the dictionary with indentation using json.dumps()
    return output_dict




In [ ]:
#@title Test the output. Note the response now uses a Json compatible template.
print(json.dumps(prompt_gpt("hi"), indent=2))

{
  "response": "Hello! How can I assist you today?"
}


# Applying LLMs to Data

In the next portion of the notebook we will examine how to apply the LLM to data. We will cover importing the data, creating a prompt, and applying the prompt to rows of data.

## Data Importation

Use data with permisive privacy/use agreements

Boder Holocaust Testimonies 2020:
https://drive.google.com/drive/folders/1VWj4p8mSZmw5DHDMVs0npZzErPD8a1y9

In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Add the Shared Folder to Your Google Drive
* Navigate to the "Shared with Me" Section in Google Drive
* Find the `Holocaust and Genocide Studies Digital Research Lab` folder in the "Shared with me" area.
* Right-click the folder and choose "Add shortcut to Drive."
* Choose the `MyDrive` folder in your Drive to save the shortcut.

This will make the folder accessible through your Google Drive interface, allowing you to point to it from Google Colab.

In [ ]:
#@title Import Necessary Data from Google Drive

import os
import pandas as pd

# Set Folder Path
folder_path = '/content/drive/MyDrive/Holocaust and Genocide Studies Digital Research Lab/Data Cleaned - Ready to Use/Boder Holocaust Testimonies 2020 '

# List all xlsx files in the folder
files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.xlsx')]

# Read each file into a DataFrame and store in a list
dataframes = [pd.read_excel(file) for file in files]

# Concatenate all DataFrames into one
df = pd.concat(dataframes, ignore_index=True)


## Working With data

In this notebook we will be using the python library pandas to organize and manipulate the data


In [ ]:
#@title Inspect Data
# make sure that it is loaded

df.rename(columns={'Unnamed: 0': 'file_part', 'IntCode': 'file_num', 'Texts': 'text'}, inplace=True)
df = df.sort_values(by=['file_num', 'file_part'])
df = df.reset_index()
df.head()

,index,file_part,file_num,Time,Names,Speaker,text,BrackInfo,IsKeep,Info
0,49907,0,1,00:00:01,Ilmar A.,David Boder,"Spool 152A, Spool 152A. Ilmar A., that seems ...","[('[', 'In English', ']')]",['N'],['In English']
1,49908,1,1,00:01:20,Ilmar A.,David Boder,"Munich, September the 24th, 1946, at the Deuts...","[('[', 'corrects himself', ']')]",['O'],[]
2,49909,2,1,00:01:50,Ilmar A.,David Boder,"So, Mr. Ilmar.","[('[', 'In German', ']')]",['N'],['In German']
3,49910,3,1,00:01:53,Ilmar A.,Ilmar A.,Yes.,[],[],[]
4,49911,4,1,00:01:54,Ilmar A.,David Boder,Let me tell you frankly : Do not tell me some...,"[('[', '?', ']'), ('[', 'possibly “we are info...","['O', 'O', 'Y']",[]


In [ ]:
# Add the is_interviwee column. David Boder is the interviewer for all of the testimonies

is_interviewee = [int(not(ele == 'David Boder')) for ele in df['Speaker']]
df['is_interviewee'] = is_interviewee

# Get the text length for each text

qword_count = []

for i in range(len(df)):
    qword_count.append(len(str(df['text'].iloc[i]).split()))

df['text_length'] = qword_count

df.head()

,index,file_part,file_num,Time,Names,Speaker,text,BrackInfo,IsKeep,Info,is_interviewee,text_length
0,49907,0,1,00:00:01,Ilmar A.,David Boder,"Spool 152A, Spool 152A. Ilmar A., that seems ...","[('[', 'In English', ']')]",['N'],['In English'],0,30
1,49908,1,1,00:01:20,Ilmar A.,David Boder,"Munich, September the 24th, 1946, at the Deuts...","[('[', 'corrects himself', ']')]",['O'],[],0,58
2,49909,2,1,00:01:50,Ilmar A.,David Boder,"So, Mr. Ilmar.","[('[', 'In German', ']')]",['N'],['In German'],0,3
3,49910,3,1,00:01:53,Ilmar A.,Ilmar A.,Yes.,[],[],[],1,1
4,49911,4,1,00:01:54,Ilmar A.,David Boder,Let me tell you frankly : Do not tell me some...,"[('[', '?', ']'), ('[', 'possibly “we are info...","['O', 'O', 'Y']",[],0,88


## Cleaning Data

Some of the texts have spools or brackets in them. We want to remove these as they are unneccessary and can confuse the model.


### Import spreadsheet with revised versions of text

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sheet_id = "18j5XIg_2XIbepIlrADsiPiVi5LWC_HXCS5xQ5v6gm2s"
worksheet_name = "Sheet1"
sh = gc.open_by_key(sheet_id)
worksheet = sh.worksheet(worksheet_name)
full_df = pd.DataFrame(worksheet.get_all_records())

### Filter for questions that need edits

In [ ]:
needs_updating = full_df[full_df['needs_edits'] == 1]
needs_updating.head()

,file_num,file_part,Time,Speaker,text,needs_edits,updated_version
0,1,4,0:01:54,David Boder,Let me tell you frankly : Do not tell me some...,1,Let me tell you frankly: Do not tell me some g...
79,1,198,0:21:42,David Boder,What happened then? [Boder as if talking to s...,1,What happened then? How did the farmer behave ...
81,1,202,0:22:10,David Boder,"And what did they you, [corrects himself] and...",1,And did they pay you anything?
218,3,191,0:29:16,David Boder,"And so, Jürgen, at first you have been doing ...",1,"And so, Jürgen, at first you have been doing ..."
486,4,542,0:23:29,David Boder,"In Hartford, Connecticut? What does he do ther...",1,"What kind of job, what does he do there?"


###Update DF with updated texts.

In [ ]:
#loops through dataframe. Finds the lines that match with the needs updating dataframe and creates a new column of all the new texts to use.

needs_edits = []
updated_text = []

for idx, row in df.iterrows():
  if row['text'] in needs_updating['text'].values:
    needs_edits.append(1)
    updated_text.append(needs_updating[needs_updating['text'] == row['text']]['updated_version'].values[0])
  else:
    needs_edits.append(0)
    updated_text.append(row['text'])

df['needs_edits'] = needs_edits
df['updated_text'] = updated_text

df.rename(columns={'text': 'original_text', 'updated_text': 'text'}, inplace=True)

# Preprocess Data

We need to prepare the data so it can be easily sent to an LLM. The first step is to assemble the necessary context for each API call. For this step, we will use tiktoken tokenize the interview, and create chunks of content that fit within a specified token limit.

In [ ]:
# Set token limit and model
token_limit = 500
model = 'gpt-4o-mini'

# Filter out NaN values in 'text' column
df_filtered = df[df['text'].notna()]

In [ ]:
#@title Import Necessary Libraries
!pip install tiktoken #allows for free token estimation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.5 MB/s eta 0:00:00


In [ ]:
#@title Define a function to preprocess the dataset by assemblying the necessary context
import pandas as pd
import tiktoken

def generate_context(df, model, token_limit):
    """
    Generate context for each row with a rolling window of the previous context,
    constrained by a token limit. The context will include text up to, but not including, the current row.

    Args:
    df (pd.DataFrame): The DataFrame containing the text data.
    model (str): The name of the LLM model for tokenization.
    token_limit (int): The maximum number of tokens allowed for the context.

    Returns:
    pd.DataFrame: A new DataFrame with an additional 'context' column.
    """
    # Initialize the tokenizer based on the provided model
    encoding = tiktoken.encoding_for_model(model)

    contexts = []
    current_context_lines = []  # Use a list to manage context lines
    current_file = None
    current_token_count = 0  # Keep track of current token count
    tokens = []

    for idx, row in df.iterrows():

        # if 'is_interviewee' = 1, speaker = 'SUBJECT'
        # if 'is_interviewee' = 0, speaker = 'INTERVIEWER'
        # if 'is_interviewee' = 2, speaker = 'CREW'

        if row['is_interviewee'] == 1:
            speaker = 'SUBJECT'
        elif row['is_interviewee'] == 0:
            speaker = 'INTERVIEWER'
        else:
            speaker = 'CREW'

        file_num = row['file_num']

        # If it's a new interview file, reset the context
        if current_file != file_num:
            current_context_lines = []
            current_token_count = 0
            current_file = file_num

        # Join the current context lines into a single string (before adding the current line)
        context = ''.join(current_context_lines)
        contexts.append(context)
        tokens.append(current_token_count)

        # Add the current line to the context AFTER the current context is saved
        current_line = f"{speaker}: {row['text']}\n"
        current_line_tokens = encoding.encode(current_line)

        # Add tokens of the new line to the total token count
        current_token_count += len(current_line_tokens)
        current_context_lines.append(current_line)

        # Trim context if token count exceeds the limit
        while current_token_count > token_limit:
            # Remove the oldest line
            if len(current_context_lines) > 1:
              removed_line = current_context_lines.pop(0)
              removed_line_tokens = encoding.encode(removed_line)
              current_token_count -= len(removed_line_tokens)

            else:
              break
    # Add the context to the dataframe
    df['context'] = contexts
    df['token_count'] = tokens
    return df

In [ ]:
#@title Create Context for the dataframe

# Generate context
context_df = generate_context(df_filtered.copy(), model, token_limit)

In [ ]:
context_df.head()

,index,file_part,file_num,Time,Names,Speaker,original_text,BrackInfo,IsKeep,Info,is_interviewee,text_length,needs_edits,text,context,token_count
0,49907,0,1,00:00:01,Ilmar A.,David Boder,"Spool 152A, Spool 152A. Ilmar A., that seems ...","[('[', 'In English', ']')]",['N'],['In English'],0,30,0,"Spool 152A, Spool 152A. Ilmar A., that seems ...",,0
1,49908,1,1,00:01:20,Ilmar A.,David Boder,"Munich, September the 24th, 1946, at the Deuts...","[('[', 'corrects himself', ']')]",['O'],[],0,58,0,"Munich, September the 24th, 1946, at the Deuts...","INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",57
2,49909,2,1,00:01:50,Ilmar A.,David Boder,"So, Mr. Ilmar.","[('[', 'In German', ']')]",['N'],['In German'],0,3,0,"So, Mr. Ilmar.","INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",148
3,49910,3,1,00:01:53,Ilmar A.,Ilmar A.,Yes.,[],[],[],1,1,0,Yes.,"INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",160
4,49911,4,1,00:01:54,Ilmar A.,David Boder,Let me tell you frankly : Do not tell me some...,"[('[', '?', ']'), ('[', 'possibly “we are info...","['O', 'O', 'Y']",[],0,88,1,Let me tell you frankly: Do not tell me some g...,"INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",165


In [ ]:
#@title Confirm that there are no long responses, which means there is no context in the suceeding text

no_tokens = context_df[(context_df['token_count'] == 0) & (context_df['file_part'] > 0)]
no_tokens

,index,file_part,file_num,Time,Names,Speaker,original_text,BrackInfo,IsKeep,Info,is_interviewee,text_length,needs_edits,text,context,token_count


# Identifying Questions

use OPENAI llm to identify which interviewer statements are questions

Warning: Takes a while, at 26000 iterations at approx 2.5 it/s, so roughly 3 hours


In [ ]:
#@title Filter for interviewer statements
int_df = context_df[context_df['is_interviewee'] == 0]
int_df.head()

,index,file_part,file_num,Time,Names,Speaker,original_text,BrackInfo,IsKeep,Info,is_interviewee,text_length,needs_edits,text,context,token_count
0,49907,0,1,00:00:01,Ilmar A.,David Boder,"Spool 152A, Spool 152A. Ilmar A., that seems ...","[('[', 'In English', ']')]",['N'],['In English'],0,30,0,"Spool 152A, Spool 152A. Ilmar A., that seems ...",,0
1,49908,1,1,00:01:20,Ilmar A.,David Boder,"Munich, September the 24th, 1946, at the Deuts...","[('[', 'corrects himself', ']')]",['O'],[],0,58,0,"Munich, September the 24th, 1946, at the Deuts...","INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",57
2,49909,2,1,00:01:50,Ilmar A.,David Boder,"So, Mr. Ilmar.","[('[', 'In German', ']')]",['N'],['In German'],0,3,0,"So, Mr. Ilmar.","INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",148
4,49911,4,1,00:01:54,Ilmar A.,David Boder,Let me tell you frankly : Do not tell me some...,"[('[', '?', ']'), ('[', 'possibly “we are info...","['O', 'O', 'Y']",[],0,88,1,Let me tell you frankly: Do not tell me some g...,"INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",165
6,49913,6,1,00:02:38,Ilmar A.,David Boder,… until today. But to you personally. Where di...,"[('[', 'Bend down to the microphone?', ']')]",['N'],['Bend down to the microphone?'],0,54,0,… until today. But to you personally. Where di...,"INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",262


In [ ]:
#@title Create structure and prompt of OpenAI response
class ResponseFormat(BaseModel):
    is_question: bool = Field(..., description="True if the Current Text is a question; False otherwise.")
    is_command: bool = Field(..., description="True if the Current Text is a command, asking the interviewee to do something; False otherwise.")

promptTemplate = 'Task: the following text is from an interview. Your job is to identify whether the text is a question, command, or neither. Text: {}'

In [ ]:
#@title Run prompt on interviewer statements

from tqdm import tqdm #just a progress bar

response_format = ResponseFormat

is_question = []
is_command = []
pbar = tqdm(int_df.iterrows())

length = len(int_df)
index = 0

# For each interviewer statement, ask GPT-4o-mini if the statement is a question or not.

for idx, row in pbar:
  index = index + 1
  text = str(row['text'])
  if '?' in text:
    is_question.append(True)
    is_command.append(False)
  else:
    prompt = promptTemplate.format(row['text'])
    response_dict = prompt_gpt(prompt, model=model, response_format=response_format)
    is_question.append(response_dict['is_question'])
    is_command.append(response_dict['is_command'])
  pbar.set_description(f"Processing row {index} of {length}")


int_df['is_question'] = is_question
int_df['is_command'] = is_command

Processing row 26493 of 26493: : 26493it [1:42:30,  4.31it/s]
<ipython-input-28-ad2e22b9bcb4>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  int_df['is_question'] = is_question
<ipython-input-28-ad2e22b9bcb4>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  int_df['is_command'] = is_command


In [ ]:
#@title Filter for interviewer questions

int_qs = int_df[(int_df['is_question'] == True) | (int_df['is_command'] == True)]
int_qs.head()

,index,file_part,file_num,Time,Names,Speaker,original_text,BrackInfo,IsKeep,Info,is_interviewee,text_length,needs_edits,text,context,token_count,is_question,is_command
4,49911,4,1,00:01:54,Ilmar A.,David Boder,Let me tell you frankly : Do not tell me some...,"[('[', '?', ']'), ('[', 'possibly “we are info...","['O', 'O', 'Y']",[],0,88,1,Let me tell you frankly: Do not tell me some g...,"INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",165,True,False
6,49913,6,1,00:02:38,Ilmar A.,David Boder,… until today. But to you personally. Where di...,"[('[', 'Bend down to the microphone?', ']')]",['N'],['Bend down to the microphone?'],0,54,0,… until today. But to you personally. Where di...,"INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",262,True,False
8,49915,8,1,00:03:19,Ilmar A.,David Boder,How old were you then? What grade were you in ...,[],[],[],0,11,0,How old were you then? What grade were you in ...,"INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",361,True,False
10,49917,10,1,00:03:31,Ilmar A.,David Boder,What means “first grade”? The highest grade?,[],[],[],0,7,0,What means “first grade”? The highest grade?,"INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",409,True,False
12,49919,12,1,00:03:38,Ilmar A.,David Boder,… of the “Volks-”?,[],[],[],0,4,0,… of the “Volks-”?,"INTERVIEWER: Spool 152A, Spool 152A. Ilmar A....",441,True,False


In [ ]:
#create and download csv file

int_qs.to_csv('int_qs.csv')
from google.colab import files


files.download('int_qs.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>